In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats

#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from sklearn.mixture import GaussianMixture 
#import testing
import sys
sys.path.append("../")
import selection_tests

In [2]:
data = pd.read_stata("all_plans_c.dta")
data['log_enroll'] = np.log(data['enr_c'])
data['enr_total'] = data['enr_c'] + data['enr_FFS']
data = data[data['year'] > 2006]

In [3]:
#https://www.statsmodels.org/dev/examples/notebooks/generated/glm_weights.html

def drop_data(data,y_name,x_name,absorb=None):
    data = data.copy()
    data = data[y_name + x_name + absorb]
    missing_vals = ~data.isnull().max(axis=1)
    data = data[missing_vals]
    data = data[data['year'].groupby(data['ssa']).transform('count')>=11]
    return data
    

def demean(y_name,x_name,data=None,absorb=None,cluster=None): 

    y,X = data[ y_name], data[ x_name ]
    
    y_dot = y.copy()
    X_dot = X.copy()
    
    ybar = y.mean()
    Xbar = X.mean()

    
    for effect in absorb:
        y_dot = y_dot - y.groupby(data[effect]).transform('mean')
        X_dot = X_dot - X.groupby(data[effect]).transform('mean') 
    
    y_dot = y_dot + ybar
    X_dot = X_dot + Xbar
    return y_dot, X_dot


In [4]:
y_name = ['bid_pub_c']
x_name = ['bmFFS','post_bmFFS', 'log_risk_pub_c','FFS_AB',"ins_parent_noSNPs",'log_risk_FFS','unemploy_rt','pc_income']
weight_names = ['enr_c_mean']
data1 = drop_data(data,y_name,x_name+weight_names,absorb=['ssa','year'])
y_dot, X_dot = demean(y_name,x_name, data=data1,absorb=['ssa','year'])
#missing_vals = ~data[y_name + x_name].isnull().max(axis=1)
#y_dot, X_dot = y_dot[missing_vals],X_dot[missing_vals]

#https://stats.stackexchange.com/questions/354689/are-frequency-weights-and-sampling-weights-in-practice-the-same-thing
weights = np.array(data1[weight_names]).reshape(y_dot.shape[0])
print(weights.shape[0])
#weights = (weights*weights.shape[0])/(weights**2).sum()
m1 = sm.GLM(y_dot,X_dot,freq_weights=weights**2)
m1_fit = m1.fit()
print(m1_fit.summary())

25542
                 Generalized Linear Model Regression Results                  
Dep. Variable:              bid_pub_c   No. Observations:                25542
Model:                            GLM   Df Residuals:            3805124493304
Model Family:                Gaussian   Df Model:                            7
Link Function:               identity   Scale:                          354.12
Method:                          IRLS   Log-Likelihood:            -1.6567e+13
Date:                Thu, 15 Dec 2022   Deviance:                   1.3475e+15
Time:                        18:29:20   Pearson chi2:                 1.35e+15
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
bmFFS                 0.6653   2

In [5]:
#print_results(['bid_pub_c'],data[data['year'] >=2011],weights=True)

# enrollment data

In [6]:
#print_results(['log_enroll'],data)